# Import libraries

In [ ]:
import sys
import glob, os
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
import matplotlib.pyplot as plt

from scipy.spatial import distance
from scipy import linalg
from scipy import signal

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA


%matplotlib inline
offline.init_notebook_mode()

In [ ]:
from keras.layers import Lambda, LSTM, RepeatVector, TimeDistributed, Dense, Dropout, Input
from keras.optimizers import Adam 
from keras.objectives import binary_crossentropy
from keras.models import Model
from keras import backend as K
from keras import Sequential
from keras.regularizers import l1, l2
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Prepare data

In [ ]:
path = '../../machine_learning/cloud_functions/data-large.csv'
data = pd.read_csv(path)
df = pd.DataFrame(data)

In [ ]:
columns = ['temporal_canny-series',
           'temporal_cross_correlation-series',
           'temporal_dct-series',
           'temporal_difference-series',
           'temporal_histogram_distance-series',
           'temporal_gaussian-series',
           'dimension',
           'attack',
           'title']


df = df[columns]
df = df.dropna()

In [ ]:
series = []
attack_ID = []
length = 70


for i, row in df.iterrows():
    time_series = np.fromstring(row['temporal_gaussian-series'].replace('[', '').replace(']', ''), 
                                                dtype=np.float, sep=' ')[:length]
    if len(time_series) < length:
        time_series = np.append(time_series, np.zeros(length - len(time_series)))

    series.append(time_series)
    if row['attack'] in ['1080p', '720p', '480p', '360p', '240p', '144p']:
        attack_ID.append(1)
    else:
        attack_ID.append(0)
        
df['series'] = series
df['attack_ID'] = attack_ID

In [ ]:
df = df.sample(frac=1)

df_0 = df[df['attack_ID'] == 0]
df_1 = df[df['attack_ID'] == 1]

df_train = df_1[:int(0.8*df_1.shape[0])]
df_test = df_1[int(0.8*df_1.shape[0]):]
df_attacks = df_0

train = np.stack(df_train['series'].to_numpy())
test = np.stack(df_test['series'].to_numpy())
attacks = np.stack(df_attacks['series'].to_numpy())

In [ ]:
# Free memory
del df, df_train, df_attacks, df_0, df_1

In [ ]:
train_ = np.copy(train)
test_ = np.copy(test)
attacks_ = np.copy(attacks)

In [ ]:
scaler = MinMaxScaler()

train_ = scaler.fit_transform(train_.reshape((train.shape[0]*train.shape[1], 1))).reshape((train.shape[0], train.shape[1]))
test_ = scaler.transform(test_.reshape((test_.shape[0]*test.shape[1], 1))).reshape((test.shape[0], test.shape[1]))
attacks_ = scaler.transform(attacks_.reshape((attacks_.shape[0]*attacks.shape[1], 1))).reshape((attacks.shape[0], attacks.shape[1]))

# Classification by thresholding the mean

In [ ]:
th = np.quantile(np.mean(train_, axis=1), 0.99)

mse_train = np.mean(train_, axis=1)
mse_test = np.mean(test_, axis=1)
mse_attacks = np.mean(attacks_, axis=1)

print('Thresholding the 99% quantile')
print('Train TPR: {}'.format(1 - sum(np.array(mse_train) > th) / len(mse_train)))
print('Test TPR: {}'.format(1 - sum(np.array(mse_test) > th) / len(mse_test)))
print('TNR: {}'.format(1 - sum(np.array(mse_attacks) < th) / len(mse_attacks)))

true_positives = sum(np.array(mse_test) < th)
false_negatives = sum(np.array(mse_test) > th)
false_positives = sum(np.array(mse_attacks) < th)
true_negatives = sum(np.array(mse_attacks) > th)

beta = 20
precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
print('F20: {}'.format(F20))

print('\n-------------------\n')

th = np.quantile(mse_test, 0.999)
print('Thresholding the 99.9% quantile')
print('Train TPR: {}'.format(1 - sum(np.array(mse_train) > th) / len(mse_train)))
print('Test TPR: {}'.format(1 - sum(np.array(mse_test) > th) / len(mse_test)))
print('TNR: {}'.format(1 - sum(np.array(mse_attacks) < th) / len(mse_attacks)))

true_positives = sum(np.array(mse_test) < th)
false_negatives = sum(np.array(mse_test) > th)
false_positives = sum(np.array(mse_attacks) < th)
true_negatives = sum(np.array(mse_attacks) > th)

beta = 20
precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
print('F20: {}'.format(F20))

# Helper functions

In [ ]:
def contractive_loss(y_pred, y_true, lam=0.00001):
    mse = K.mean(K.square(y_true - y_pred), axis=1)

    W = K.variable(value=model.get_layer('encoded').get_weights()[0])  # N x N_hidden
    W = K.transpose(W)  # N_hidden x N
    h = model.get_layer('encoded').output
    dh = h * (1 - h)  # N_batch x N_hidden, derivative of sigmoid function
    contractive = lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)

    return mse + contractive


def plot_history(network_history, title):
    plt.figure(figsize=(10, 5))
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.semilogy(network_history.history['loss'])
    plt.semilogy(network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])
    plt.grid()
    plt.show()
    
def print_scores(mse_train, mse_test, mse_attacks):
    th = np.quantile(np.mean(train_, axis=1), 0.99)

    mse_train = np.mean(train_, axis=1)
    mse_test = np.mean(test_, axis=1)
    mse_attacks = np.mean(attacks_, axis=1)

    print('Thresholding the 99% quantile')
    print('Train TPR: {}'.format(1 - sum(np.array(mse_train) > th) / len(mse_train)))
    print('Test TPR: {}'.format(1 - sum(np.array(mse_test) > th) / len(mse_test)))
    print('TNR: {}'.format(1 - sum(np.array(mse_attacks) < th) / len(mse_attacks)))

    true_positives = sum(np.array(mse_test) < th)
    false_negatives = sum(np.array(mse_test) > th)
    false_positives = sum(np.array(mse_attacks) < th)
    true_negatives = sum(np.array(mse_attacks) > th)

    beta = 20
    precision = true_positives/(true_positives+false_positives)
    recall = true_positives/(true_positives+false_negatives)
    F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
    print('F20: {}'.format(F20))

    print('\n-------------------\n')

    th = np.quantile(mse_test, 0.999)
    print('Thresholding the 99.9% quantile')
    print('Train TPR: {}'.format(1 - sum(np.array(mse_train) > th) / len(mse_train)))
    print('Test TPR: {}'.format(1 - sum(np.array(mse_test) > th) / len(mse_test)))
    print('TNR: {}'.format(1 - sum(np.array(mse_attacks) < th) / len(mse_attacks)))

    true_positives = sum(np.array(mse_test) < th)
    false_negatives = sum(np.array(mse_test) > th)
    false_positives = sum(np.array(mse_attacks) < th)
    true_negatives = sum(np.array(mse_attacks) > th)

    beta = 20
    precision = true_positives/(true_positives+false_positives)
    recall = true_positives/(true_positives+false_negatives)
    F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
    print('F20: {}'.format(F20))
    
def print_series(train_, train_re, test_, test_re, attacks_, attacks_re):
    
    f, axs = plt.subplots(3,5,figsize=(20,15))

    ims = np.random.randint(0,3000,5)

    axs[0,0].plot(train_[ims[0]])
    axs[0,0].plot(train_re[ims[0]])

    axs[0,1].plot(train_[ims[1]])
    axs[0,1].plot(train_re[ims[1]])

    axs[0,2].plot(train_[ims[2]])
    axs[0,2].plot(train_re[ims[2]])

    axs[0,3].plot(train_[ims[3]])
    axs[0,3].plot(train_re[ims[3]])

    axs[0,4].plot(train_[ims[4]])
    axs[0,4].plot(train_re[ims[4]])


    axs[1,0].plot(test_[ims[0]])
    axs[1,0].plot(test_re[ims[0]])

    axs[1,1].plot(test_[ims[1]])
    axs[1,1].plot(test_re[ims[1]])

    axs[1,2].plot(test_[ims[2]])
    axs[1,2].plot(test_re[ims[2]])

    axs[1,3].plot(test_[ims[3]])
    axs[1,3].plot(test_re[ims[3]])

    axs[1,4].plot(test_[ims[4]])
    axs[1,4].plot(test_re[ims[4]])


    axs[2,0].plot(attacks_[ims[0]])
    axs[2,0].plot(attacks_re[ims[0]])

    axs[2,1].plot(attacks_[ims[1]])
    axs[2,1].plot(attacks_re[ims[1]])

    axs[2,2].plot(attacks_[ims[2]])
    axs[2,2].plot(attacks_re[ims[2]])

    axs[2,3].plot(attacks_[ims[3]])
    axs[2,3].plot(attacks_re[ims[3]])

    axs[2,4].plot(attacks_[ims[4]])
    axs[2,4].plot(attacks_re[ims[4]])

# Autoencoder

In [ ]:
n_in = train_.shape[1]
latent_dim = 10
model = Sequential()
model.add(Dense(8 * latent_dim, activation='relu', input_shape=(n_in,)))
model.add(Dropout(0.1))
model.add(Dense(latent_dim, activation='linear', name='encoded'))
model.add(Dense(8 * latent_dim, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(n_in, activation='tanh'))
adam = Adam(lr=0.0001)

In [ ]:
model.compile(optimizer=adam, loss='mae')
model.summary()
history = model.fit(train_, train_, epochs=450, verbose=0, 
                    batch_size=128, validation_data=(test_, test_), shuffle=True)

plot_history(history, 'AE')

In [ ]:
train_re = model.predict(train_, batch_size=2048)
test_re = model.predict(test_, batch_size=2048)
attacks_re = model.predict(attacks_, batch_size=2048)

In [ ]:
mse_train = [mean_squared_error(train_[i], train_re[i]) for i,_ in enumerate(train_)]
mse_test = [mean_squared_error(test_[i], test_re[i]) for i, _ in enumerate(test_)]
mse_attacks = [mean_squared_error(attacks_[i], attacks_re[i]) for i, _ in enumerate(attacks_)]

In [ ]:
np.mean(mse_train), np.mean(mse_test), np.mean(mse_attacks)

In [ ]:
np.std(mse_train), np.std(mse_test), np.std(mse_attacks)

In [ ]:
print_scores(mse_train, mse_test, mse_attacks)

In [ ]:
print_series(train_, train_re, test_, test_re, attacks_, attacks_re)

## Study latent space and classify with OCSVM

In [ ]:
encoder = K.function([model.layers[0].input], [model.get_layer('encoded').output],)

In [ ]:
train_red = encoder([train_])[0]
test_red = encoder([test_])[0]
attacks_red = encoder([attacks_])[0]

In [ ]:
ss = StandardScaler()

train_red_ = ss.fit_transform(train_red)
test_red_ = ss.transform(test_red)
attacks_red_ = ss.transform(attacks_red)

variances = []
components = reversed(range(1,train_red_.shape[1]+1))
for i in components:
    pca = PCA(n_components=i)
    pca.fit(train_red_)
    variances.append(sum(pca.explained_variance_ratio_))
    
trace = go.Scatter(
x = list(reversed(range(1,test_red_.shape[1]+1))),
y = variances)

data=[trace]

layout = {'title': 'PCA', 
      'xaxis': {'title': 'Number of components', }, 
      'yaxis': {'title': 'Variance explained'},
      }

fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(train_red)
test_reduced = pca.transform(test_red)
attack_reduced = pca.transform(attacks_red)

f, ax = plt.subplots(1,3, figsize=(25,10))
ax[0].set_title("Train set")
ax[1].set_title("Test set")
ax[2].set_title("Attack set")
ax[0].scatter(X_reduced[:,0], X_reduced[:,1], color='black')
ax[1].scatter(test_reduced[:,0], test_reduced[:,1], color='red')
ax[2].scatter(attack_reduced[:,0], attack_reduced[:,1], color='blue')

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(attack_reduced[:,0], attack_reduced[:,1], color='red', label='attack')
plt.scatter(X_reduced[:,0], X_reduced[:,1], color='green', label='Train')
plt.scatter(test_reduced[:,0], test_reduced[:,1], color='yellow', label='Test')
plt.legend()

In [ ]:
OCSVM = svm.OneClassSVM(kernel='rbf',gamma='auto', nu=0.001, cache_size=7000, tol=0.00001, shrinking=False)
OCSVM.fit(train_red)

true_positives = sum(OCSVM.predict(test_red) == 1)
false_negatives = sum(OCSVM.predict(test_red) == -1)
false_positives = sum(OCSVM.predict(attacks_red) == 1)
true_negatives = sum(OCSVM.predict(attacks_red) == -1)

beta = 20
precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
print('F20: {}'.format(F20))
print('Test TPR: {}'.format(true_positives/len(test_red)))
print('TNR: {}'.format(true_negatives/len(attacks_red)))

In [ ]:
# Classify by latent space uniqueness
ids = np.unique(np.argmax(train_red, axis=1))
ids, np.unique(np.argmax(test_red, axis=1)), np.unique(np.argmax(attacks_red, axis=1))

In [ ]:
test_pred = [1 if i in ids else 0 for i in np.argmax(test_red, axis=1)]
attack_pred = [1 if i in ids else 0 for i in np.argmax(attacks_red, axis=1)]

true_positives = sum(np.asarray(test_pred) == 1)
false_negatives = sum(np.asarray(test_pred) == 0)
false_positives = sum(np.asarray(attack_pred) == 1)
true_negatives = sum(np.asarray(attack_pred) == 0)

beta = 20
precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
print('F20: {}'.format(F20))
print('Test TPR: {}'.format(true_positives/len(test_red)))
print('TNR: {}'.format(true_negatives/len(attacks_red)))

# Two-Input Autoencoder

In [ ]:
path = '../../machine_learning/cloud_functions/data-large.csv'
data = pd.read_csv(path)
df = pd.DataFrame(data)

In [ ]:
columns = ['temporal_canny-series',
           'temporal_cross_correlation-series',
           'temporal_dct-series',
           'temporal_difference-series',
           'temporal_histogram_distance-series',
           'temporal_gaussian-series',
           'dimension',
           'attack',
           'title']


df = df[columns]
df = df.dropna()

In [ ]:
series = []
series_1 = []
attack_ID = []
length = 70


for i, row in df.iterrows():
    time_series = row['dimension']*np.fromstring(row['temporal_dct-series'].replace('[', '').replace(']', ''), 
                                                dtype=np.float, sep=' ')[:length]
    time_series_1 = np.fromstring(row['temporal_gaussian-series'].replace('[', '').replace(']', ''), 
                                            dtype=np.float, sep=' ')[:length]
    if len(time_series) < length:
        time_series = np.append(time_series, np.zeros(length - len(time_series)))
        
    if len(time_series_1) < length: 
        time_series_1 = np.append(time_series_1, np.zeros(length - len(time_series_1)))
        
    series.append(time_series)
    series_1.append(time_series_1)
    if row['attack'] in ['1080p', '720p', '480p', '360p', '240p', '144p']:
        attack_ID.append(1)
    else:
        attack_ID.append(0)
        
df['series'] = series
df['series_1'] = series_1
df['attack_ID'] = attack_ID

In [ ]:
df = df.sample(frac=1)

df_0 = df[df['attack_ID'] == 0]
df_1 = df[df['attack_ID'] == 1]

df_train = df_1[:int(0.8*df_1.shape[0])]
df_test = df_1[int(0.8*df_1.shape[0]):]
df_attacks = df_0

train = np.stack(df_train['series'].to_numpy())
test = np.stack(df_test['series'].to_numpy())
attacks = np.stack(df_attacks['series'].to_numpy())

train_1 = np.stack(df_train['series_1'].to_numpy())
test_1 = np.stack(df_test['series_1'].to_numpy())
attacks_1 = np.stack(df_attacks['series_1'].to_numpy())


In [ ]:
# Free memory
del df, df_train, df_attacks, df_0, df_1

In [ ]:
train_ = np.copy(train)
test_ = np.copy(test)
attacks_ = np.copy(attacks)

train_1 = np.copy(train_1)
test_1 = np.copy(test_1)
attacks_1 = np.copy(attacks_1)

In [ ]:
scaler = MinMaxScaler()
# scaler = StandardScaler()


train_ = scaler.fit_transform(train_.reshape((train.size, 1))).reshape((train.shape[0],
                                                                        train.shape[1]))
test_ = scaler.transform(test_.reshape((test.size, 1))).reshape((test.shape[0],
                                                                 test.shape[1]))
attacks_ = scaler.transform(attacks_.reshape((attacks.size, 1))).reshape((attacks.shape[0],
                                                                          attacks.shape[1]))

scaler = MinMaxScaler()
# scaler = StandardScaler()


train_1_ = scaler.fit_transform(train_1.reshape((train.size, 1))).reshape((train.shape[0],
                                                                        train.shape[1]))
test_1_ = scaler.transform(test_1.reshape((test.size, 1))).reshape((test.shape[0],
                                                                 test.shape[1]))
attacks_1_ = scaler.transform(attacks_1.reshape((attacks.size, 1))).reshape((attacks.shape[0],
                                                                          attacks.shape[1]))

In [ ]:
train = np.hstack((train_, train_1_))
test = np.hstack((test_, test_1_))
attacks = np.hstack((attacks_, attacks_1_))

In [ ]:
train__ = train_ + np.random.normal(scale=0.005, size=(train_.shape[0], train_.shape[1]))
test__ = test_ + np.random.normal(scale=0.005, size=(test_.shape[0], test_.shape[1]))
attacks__ = attacks_ + np.random.normal(scale=0.005, size=(attacks_.shape[0], attacks_.shape[1]))

In [ ]:
n_in = train_.shape[1]
latent_dim = 30
model = Sequential()
model.add(Dense(8 * latent_dim, activation='relu', input_shape=(n_in,)))
model.add(Dropout(0.2))
model.add(Dense(latent_dim, activation='linear', name='encoded'))
model.add(Dense(8 * latent_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(n_in, activation='linear'))
adam = Adam(lr=0.0001)

In [ ]:
model.compile(optimizer=adam, loss='mae')
model.summary()
history = model.fit(train__, train_, epochs=150, verbose=0, 
                    batch_size=128, validation_data=(test__, test_), shuffle=True)

In [ ]:
plot_history(history, 'AE')

In [ ]:
train_re = model.predict(train__, batch_size=2048)
test_re = model.predict(test__, batch_size=2048)
attacks_re = model.predict(attacks__, batch_size=2048)

In [ ]:
mse_train = [mean_squared_error(train_[i], train_re[i]) for i,_ in enumerate(train_)]
mse_test = [mean_squared_error(test_[i], test_re[i]) for i, _ in enumerate(test_)]
mse_attacks = [mean_squared_error(attacks_[i], attacks_re[i]) for i, _ in enumerate(attacks_)]

In [ ]:
np.mean(mse_train), np.mean(mse_test), np.mean(mse_attacks)

In [ ]:
np.std(mse_train), np.std(mse_test), np.std(mse_attacks)

In [ ]:
print_scores(mse_train, mse_test, mse_attacks)

In [ ]:
print_series(train_, train_re, test_, test_re, attacks_, attacks_re)

## Study latent space and classify with OCSVM

In [ ]:
encoder = K.function([model.layers[0].input], [model.get_layer('encoded').output],)

In [ ]:
train_red = encoder([train_])[0]
test_red = encoder([test_])[0]
attacks_red = encoder([attacks_])[0]

In [ ]:
ss = StandardScaler()

train_red_ = ss.fit_transform(train_red)
test_red_ = ss.transform(test_red)
attacks_red_ = ss.transform(attacks_red)

variances = []
components = reversed(range(1,train_red_.shape[1]+1))
for i in components:
    pca = PCA(n_components=i)
    pca.fit(train_red_)
    variances.append(sum(pca.explained_variance_ratio_))
    
trace = go.Scatter(
x = list(reversed(range(1,test_red_.shape[1]+1))),
y = variances)

data=[trace]

layout = {'title': 'PCA', 
      'xaxis': {'title': 'Number of components', }, 
      'yaxis': {'title': 'Variance explained'},
      }

fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(train_red)
test_reduced = pca.transform(test_red)
attack_reduced = pca.transform(attacks_red)

f, ax = plt.subplots(1,3, figsize=(25,10))
ax[0].set_title("Train set")
ax[1].set_title("Test set")
ax[2].set_title("Attack set")
ax[0].scatter(X_reduced[:,0], X_reduced[:,1], color='black')
ax[1].scatter(test_reduced[:,0], test_reduced[:,1], color='red')
ax[2].scatter(attack_reduced[:,0], attack_reduced[:,1], color='blue')

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(attack_reduced[:,0], attack_reduced[:,1], color='red', label='attack')
plt.scatter(X_reduced[:,0], X_reduced[:,1], color='green', label='Train')
plt.scatter(test_reduced[:,0], test_reduced[:,1], color='yellow', label='Test')
plt.legend()

In [ ]:
OCSVM = svm.OneClassSVM(kernel='rbf',gamma='auto', nu=0.001, cache_size=7000, tol=0.00001, shrinking=False)
OCSVM.fit(train_red)

true_positives = sum(OCSVM.predict(test_red) == 1)
false_negatives = sum(OCSVM.predict(test_red) == -1)
false_positives = sum(OCSVM.predict(attacks_red) == 1)
true_negatives = sum(OCSVM.predict(attacks_red) == -1)

beta = 20
precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
print('F20: {}'.format(F20))
print('Test TPR: {}'.format(true_positives/len(test_red)))
print('TNR: {}'.format(true_negatives/len(attacks_red)))

In [ ]:
# Classify by latent space uniqueness
ids = np.unique(np.argmax(train_red, axis=1))
ids, np.unique(np.argmax(test_red, axis=1)), np.unique(np.argmax(attacks_red, axis=1))

In [ ]:
test_pred = [1 if i in ids else 0 for i in np.argmax(test_red, axis=1)]
attack_pred = [1 if i in ids else 0 for i in np.argmax(attacks_red, axis=1)]

true_positives = sum(np.asarray(test_pred) == 1)
false_negatives = sum(np.asarray(test_pred) == 0)
false_positives = sum(np.asarray(attack_pred) == 1)
true_negatives = sum(np.asarray(attack_pred) == 0)

beta = 20
precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
F20 = (1 + (beta ** 2))*precision*recall/((beta ** 2)*precision + recall)
print('F20: {}'.format(F20))
print('Test TPR: {}'.format(true_positives/len(test_red)))
print('TNR: {}'.format(true_negatives/len(attacks_red)))

# LSTM Autoencoder

In [ ]:
train_ = train_.reshape(train.shape[0], train.shape[1], 1)
test_ = test_.reshape(test.shape[0], test.shape[1], 1)
attacks_ = attacks_.reshape(attacks.shape[0], attacks.shape[1], 1)

In [ ]:
model = Sequential()
model.add(LSTM(32, activation='relu', input_shape=(train.shape[1], 1), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(RepeatVector(train.shape[1]))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='mae')
model.summary()

In [ ]:
history = model.fit(train_, train_, epochs=150, verbose=1, 
                    batch_size=512, validation_data=(test_, test_), shuffle=True)

In [ ]:
plot_history(history, 'AE')

In [ ]:
plot_history(history, 'AE')

In [ ]:
train_re = model.predict(train_, batch_size=2048)
test_re = model.predict(test_, batch_size=2048)
attacks_re = model.predict(attacks_, batch_size=2048)

In [ ]:
mse_train = [mean_squared_error(train_[i], train_re[i]) for i,_ in enumerate(train_)]
mse_test = [mean_squared_error(test_[i], test_re[i]) for i, _ in enumerate(test_)]
mse_attacks = [mean_squared_error(attacks_[i], attacks_re[i]) for i, _ in enumerate(attacks_)]

In [ ]:
np.mean(mse_train), np.mean(mse_test), np.mean(mse_attacks)

In [ ]:
np.std(mse_train), np.std(mse_test), np.std(mse_attacks)

In [ ]:
print_scores(mse_train, mse_test, mse_attacks)

In [ ]:
print_series(train_, train_re, test_, test_re, attacks_, attacks_re)